In [2]:
import numpy as np

# Загрузка данных из файла .npy
data = np.load('ihb.npy')  # Замените 'data.npy' на путь к вашему файлу

# Проверяем форму данных
print(f"Форма исходных данных: {data.shape}")  # Должно быть [320, 10, 246]

# Усредняем по второму измерению (по временным шагам), игнорируя NaN
averaged_data = np.nanmean(data, axis=1)

# Проверяем форму после усреднения
print(f"Форма данных после усреднения: {averaged_data.shape}")  # Должно быть [320, 246]

# Сохраняем преобразованные данные, если необходимо
np.save('averaged_data.npy', averaged_data)

Форма исходных данных: (320, 10, 246)
Форма данных после усреднения: (320, 246)


C:\Users\valik\AppData\Local\Temp\ipykernel_18472\3124052259.py:10: RuntimeWarning: Mean of empty slice
  averaged_data = np.nanmean(data, axis=1)


In [3]:
import numpy as np
from itertools import combinations

# Загружаем усредненные данные (после предыдущего шага)
averaged_data = np.load('averaged_data.npy')  # Форма данных: [320, 246]

# Количество субъектов и представлений
num_subjects = 20
representations_per_subject = 16

# Генерация всех возможных пар индексов (combinations даёт все возможные пары)
pairs = list(combinations(range(averaged_data.shape[0]), 2))

# Создание массива признаков для пар объектов
train_x = []
train_y = []

for pair in pairs:
    idx1, idx2 = pair
    
    # Добавляем признаки обеих пар в массив
    train_x.append([averaged_data[idx1], averaged_data[idx2]])
    
    # Определяем метку: 1, если оба объекта принадлежат одному субъекту, 0 — если нет
    label = 1 if idx1 // representations_per_subject == idx2 // representations_per_subject else 0
    train_y.append(label)

# Преобразуем train_x и train_y в numpy массивы для дальнейшего использования
train_x = np.array(train_x)
train_y = np.array(train_y)

# Проверяем формы данных
print(f"Форма train_x: {train_x.shape}")  # Ожидаемая форма: [num_pairs, 2, 246]
print(f"Форма train_y: {train_y.shape}")  # Ожидаемая форма: [num_pairs]

# Сохранение данных, если необходимо
np.save('train_x.npy', train_x)
np.save('train_y.npy', train_y)

Форма train_x: (51040, 2, 246)
Форма train_y: (51040,)


In [7]:
# Пример ручной проверки пары
def manual_check_pair(idx1, idx2, train_x, train_y):
    # Определяем пациентов для объектов
    patient1 = idx1 // 16
    patient2 = idx2 // 16

    # Проверяем бинарную метку
    correct_label = 1 if patient1 == patient2 else 0
    actual_label = train_y[list(combinations(range(320), 2)).index((idx1, idx2))]

    print(f"Проверка пары ({idx1}, {idx2}):")
    print(f"Пациент 1: {patient1}, Пациент 2: {patient2}")
    print(f"Ожидаемая метка: {correct_label}, Фактическая метка: {actual_label}")

# Пример проверки: проверяем, принадлежат ли объекты 0 и 15 одному пациенту
manual_check_pair(0, 15, train_x, train_y)

# Пример проверки: проверяем, принадлежат ли объекты 0 и 20 разным пациентам
manual_check_pair(17, 23, train_x, train_y)


Проверка пары (0, 15):
Пациент 1: 0, Пациент 2: 0
Ожидаемая метка: 1, Фактическая метка: 1
Проверка пары (17, 23):
Пациент 1: 1, Пациент 2: 1
Ожидаемая метка: 1, Фактическая метка: 1


In [66]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Загрузка данных
train_x = np.load('train_x.npy')  # [num_pairs, 2, 246]
train_y = np.load('train_y.npy')  # [num_pairs]

# Заменим NaN на нули в train_x
train_x = np.nan_to_num(train_x, nan=0.0)  # Заменяем NaN на 0.0
# Если хотите заменить NaN на средние значения по признакам:
# mean_vals = np.nanmean(train_x, axis=0)  # Считаем средние значения по каждому признаку
# np.where(np.isnan(train_x), mean_vals, train_x)  # Заменяем NaN на средние значения
# Преобразование данных в тензоры PyTorch
train_x_tensor = torch.tensor(train_x, dtype=torch.float32)  # Признаки пар
train_y_tensor = torch.tensor(train_y, dtype=torch.float32)  # Метки (0 или 1)

# Соединяем два набора признаков в один тензор для каждой пары (вектор размером 492)
train_x_combined = torch.cat((train_x_tensor[:, 0, :], train_x_tensor[:, 1, :]), dim=1)

# Создание DataLoader для удобной подачи данных в нейронную сеть
dataset = TensorDataset(train_x_combined, train_y_tensor)
train_size = int(0.8 * len(dataset))  # 80% данных для тренировки
test_size = len(dataset) - train_size  # 20% данных для теста
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [67]:
# Проверка наличия NaN в train_x
if torch.isnan(train_x_tensor).any():
    print("В данных train_x есть NaN-значения")
else:
    print("В данных train_x нет NaN-значений")

# Проверка наличия NaN в train_y
if torch.isnan(train_y_tensor).any():
    print("В данных train_y есть NaN-значения")
else:
    print("В данных train_y нет NaN-значений")

В данных train_x нет NaN-значений
В данных train_y нет NaN-значений


In [51]:
# Определение архитектуры нейронной сети
class MatchingNetwork(nn.Module):
    def __init__(self):
        super(MatchingNetwork, self).__init__()
        # Входной слой на 492 нейрона
        self.fc1 = nn.Linear(246*2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.LeakyReLU(negative_slope=0.01)
    
    def forward(self, x):
        # Прямое распространение через сеть
        x = self.relu(self.fc1(x))  # Первый полносвязанный слой
        x = self.relu(self.fc2(x))  # Второй полносвязанный слой
        x = self.fc3(x)  # Выходной слой с сигмоидальной активацией
        return x

# Инициализируем модель
model = MatchingNetwork()

In [53]:
# Функция потерь — бинарная кросс-энтропия
criterion = nn.BCEWithLogitsLoss()
# Оптимизатор — Adam
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [18]:
max(train_y)

1

In [54]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Прямой проход (forward)
            outputs = model(inputs).squeeze()  # Убираем лишние размеры

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Ограничение норм

            # Так как последний слой уже содержит Sigmoid, можем сразу считать потери
            loss = criterion(outputs, labels)
            
            # Обратное распространение (backward) и шаг оптимизации
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss / len(train_loader)}")

# Обучаем модель
train_model(model, train_dataset, criterion, optimizer, num_epochs=10)

Эпоха [1/10], Потери: 0.18641902305854363
Эпоха [2/10], Потери: 0.1690491731731122
Эпоха [3/10], Потери: 0.15699812350648698
Эпоха [4/10], Потери: 0.14288729506283473
Эпоха [5/10], Потери: 0.12423709787842219
Эпоха [6/10], Потери: 0.10438773821676811
Эпоха [7/10], Потери: 0.08565785772159601
Эпоха [8/10], Потери: 0.06868122605389954
Эпоха [9/10], Потери: 0.0555355278435409
Эпоха [10/10], Потери: 0.04707531487272295


In [79]:
# Создание DataLoader для тренировки и теста
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

def test_model(model, test_loader):
    model.eval()  # Переводим модель в режим оценки (без обновления градиентов)
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs).squeeze()
            predictions = (outputs > 0.5).float()  # Преобразуем вероятности в бинарные метки (0 или 1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

# Запуск теста:
test_model(model, test_loader)

Accuracy: 4.67%


0.046728056426332286

Реальная метка (0 или 1, для того, что принадлежат ли они одному человеку): True
Предсказанная метка модели: tensor([[nan]])


In [62]:
torch.save(model, "D:\VS CODE\Works\YandexContest\\model.pt")